In [39]:
import pandas as pd
import numpy as np
import StringIO
import urllib
import random
import datetime
import sys
import time

pd.set_option('max_colwidth', 5000)

HOST = 'http://mtgiga001-1.metrika.yandex.net:8123'
import requests
def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    r = requests.post(host, data =  query, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

def df_to_wiki(df, show_index = False):
    print '#|\n ||' + df.to_csv(index = show_index, sep = '|').replace('|', ' | ').replace('\n', '|| \n ||')[:-2] + '|#'

## Query 1

In [2]:
q1_tmpl = '''
SELECT
    sum(Sign),
    SearchEngineID,
    StartDate
FROM {table}
WHERE (StartDate >= toDate('{date}')) AND (StartDate <= toDate('{date}'))
    AND (regionToCountry(RegionID) = 983) AND (IsYandex = 0) AND (IsMobile = 0) AND (UserAgent = 70)
    AND NOT (SearchEngineID IN (0, 74))
GROUP BY
    SearchEngineID,
    StartDate
'''

In [42]:
q1_stats = []

def execute_query(query_tmpl, date, table, query_num):
    time1 = time.time()
    tmp = get_clickhouse_data(
        query_tmpl.format(
            date = date.strftime('%Y-%m-%d'),
            table = table
        )
    )
    time2 = time.time()
    
    item = {
        'date': date.strftime('%Y-%m-%d'),
        'table': 'counter_id' if 'zgmnkv_counter_key' in table else 'wo_counter_id',
        'timing': time2 - time1,
        'query': query_num
    }
    return item

for date in pd.date_range('2017-10-08', '2017-10-14'):
    for table in ['visits_all', 'zgmnkv_counter_key.visits_all']:
        q1_stats.append(execute_query(q1_tmpl, date, table, 1))

In [43]:
def show_stats(stats):
    stats_df = pd.DataFrame(stats).pivot_table(index = 'date', columns = 'table', values = 'timing')
    stats_df['share'] = stats_df.counter_id/stats_df.wo_counter_id
    print 'median', round(stats_df.share.median(), 2)
    
    return stats_df

In [44]:
show_stats(q1_stats)

median 3.94


table,counter_id,wo_counter_id,share
date,,,
2017-10-08,4.212165,1.070023,3.936518
2017-10-09,4.246920,1.093207,3.884827
2017-10-10,3.481951,1.418156,2.455267
2017-10-11,3.151289,0.711623,4.428313
2017-10-12,3.317952,0.699189,4.745428
2017-10-13,3.902505,1.015923,3.841340
2017-10-14,3.478346,0.820349,4.240081


In [45]:
df_to_wiki(show_stats(q1_stats).applymap(lambda x: round(x, 2)), True)

median 3.94
#|
 ||date | counter_id | wo_counter_id | share|| 
 ||2017-10-08 | 4.21 | 1.07 | 3.94|| 
 ||2017-10-09 | 4.25 | 1.09 | 3.88|| 
 ||2017-10-10 | 3.48 | 1.42 | 2.46|| 
 ||2017-10-11 | 3.15 | 0.71 | 4.43|| 
 ||2017-10-12 | 3.32 | 0.7 | 4.75|| 
 ||2017-10-13 | 3.9 | 1.02 | 3.84|| 
 ||2017-10-14 | 3.48 | 0.82 | 4.24|| 
 |#


## Query 2

In [8]:
q2_tmpl = '''
SELECT
    sum(Sign),
    SearchEngineID,
    StartDate,
    IsMobile,
    (OSToRoot(OS) = 108) as is_android,
    ((UserAgent = 83) AND (UserAgentMajor = 45)) as is_chrome_mobile,
    ((UserAgent = 83) AND (UserAgentMajor >= 46) AND (UserAgentMajor <= 60)) as is_chrome_mobile2,
    (UserAgent = 83) is_mobile_chrome_br,
    ((IsMobile = 1) AND (IsTablet = 1)) as is_tablet
FROM {table}
WHERE (StartDate >= toDate('{date}')) AND (StartDate <= toDate('{date}'))
    AND (regionToCountry(RegionID) = 187) AND (IsYandex = 0)
    AND NOT (SearchEngineID IN (0, 74)) AND NOT regionIn(RegionID, toUInt32(977))
GROUP BY
    SearchEngineID,
    StartDate,
    IsMobile,
    is_android,
    is_chrome_mobile,
    is_chrome_mobile2,
    is_tablet, is_mobile_chrome_br
'''

In [9]:
%%time
q2_stats = []

for date in pd.date_range('2017-10-08', '2017-10-14'):
    for table in ['visits_all', 'zgmnkv_counter_key.visits_all']:
        q2_stats.append(execute_query(q2_tmpl, date, table, 2))

CPU times: user 44 ms, sys: 16 ms, total: 60 ms
Wall time: 39.1 s


In [10]:
show_stats(q2_stats)

median 3.61


table,counter_id,wo_counter_id,share
date,,,
2017-10-08,4.732277,1.361340,3.476190
2017-10-09,4.287912,1.409727,3.041662
2017-10-10,4.405503,1.264052,3.485222
2017-10-11,4.387425,0.830600,5.282236
2017-10-12,4.448285,0.865257,5.140998
2017-10-13,4.143320,1.147019,3.612251
2017-10-14,4.978596,0.866245,5.747330


In [46]:
df_to_wiki(show_stats(q2_stats).applymap(lambda x: round(x, 2)), True)

median 3.61
#|
 ||date | counter_id | wo_counter_id | share|| 
 ||2017-10-08 | 4.73 | 1.36 | 3.48|| 
 ||2017-10-09 | 4.29 | 1.41 | 3.04|| 
 ||2017-10-10 | 4.41 | 1.26 | 3.49|| 
 ||2017-10-11 | 4.39 | 0.83 | 5.28|| 
 ||2017-10-12 | 4.45 | 0.87 | 5.14|| 
 ||2017-10-13 | 4.14 | 1.15 | 3.61|| 
 ||2017-10-14 | 4.98 | 0.87 | 5.75|| 
 |#


## Query 3

In [48]:
q3_tmpl = '''
SELECT   
   sum(Sign) as visits,
   StartDate as date,
   if(SearchEngineID IN (0, 74), 'not_search', if(SearchEngineID IN (2, 13, 181), 'yandex', if(SearchEngineID IN (3, 68, 224), 'google', 'other'))) as se,
   StartDate as date
FROM {table}
WHERE (StartDate = toDate('{date}'))
  AND (regionToCountry(RegionID) == (225))
  AND (IsMobile = 0)
  AND SearchEngineID NOT IN (0, 74)
  AND StartURLDomain like '%baskino.co%'
GROUP BY
    date, se
 FORMAT JSONCompact
'''

In [49]:
%%time
q3_stats = []

for date in pd.date_range('2017-10-08', '2017-10-14'):
    for table in ['visits_all', 'zgmnkv_counter_key.visits_all']:
        q3_stats.append(execute_query(q3_tmpl, date, table, 3))

CPU times: user 48 ms, sys: 8 ms, total: 56 ms
Wall time: 40.1 s


In [50]:
show_stats(q3_stats)

median 5.51


table,counter_id,wo_counter_id,share
date,,,
2017-10-08,4.711743,0.817254,5.765335
2017-10-09,5.091550,1.005312,5.064647
2017-10-10,4.827339,1.042964,4.628481
2017-10-11,4.256306,0.928865,4.582266
2017-10-12,5.867202,0.933885,6.282574
2017-10-13,4.370586,0.792961,5.511728
2017-10-14,4.681950,0.801974,5.838032


In [51]:
df_to_wiki(show_stats(q3_stats).applymap(lambda x: round(x, 2)), True)

median 5.51
#|
 ||date | counter_id | wo_counter_id | share|| 
 ||2017-10-08 | 4.71 | 0.82 | 5.77|| 
 ||2017-10-09 | 5.09 | 1.01 | 5.06|| 
 ||2017-10-10 | 4.83 | 1.04 | 4.63|| 
 ||2017-10-11 | 4.26 | 0.93 | 4.58|| 
 ||2017-10-12 | 5.87 | 0.93 | 6.28|| 
 ||2017-10-13 | 4.37 | 0.79 | 5.51|| 
 ||2017-10-14 | 4.68 | 0.8 | 5.84|| 
 |#


## Query 4

In [59]:
q4_tmpl = '''
SELECT 
    user_id,
    browser,
    sum(visits) as total_visits,
    sumIf(visits, day_of_week = 'weekend') as wknd_total_visits, 
    sumIf(visits, day_of_week = 'working') as wrkng_total_visits,
    sumIf(visits, (day_of_week = 'working') AND (pod = 2)) as wrkng_pod_2_visits
FROM
    (SELECT 
        sum(Sign) as visits,
        if((toHour(ClientEventTime) as hour) < 7, 1, if(hour < 19, 2, 1)) as pod,
        if(toDayOfWeek(StartDate) IN (6, 7), 'weekend', 'working') as day_of_week,
        UserID as user_id,
        transform(UserAgent, [5, 135, 70, 6, 3, 2],
            ['MSIE', 'Edge', 'Yandex.Browser', 'Google Chrome', 'Firefox', 'Opera'],
            'other') as browser
    FROM {table} SAMPLE 1/50 OFFSET 0/50
    WHERE (StartDate <= '{date}') AND (StartDate >= toDate('{date}') - 13)
        AND (regionToCountry(RegionID) = 225)
        AND (ClientTimeZone != -1)
        AND (IsMobile = 0)
    GROUP BY 
        pod,
        user_id, 
        day_of_week,
        browser)
GROUP BY 
    user_id,
    browser
HAVING total_visits >= 100 
'''

In [61]:
%%time
q4_stats = []

for date in pd.date_range('2017-10-14', '2017-10-20'):
    for table in ['visits_all', 'zgmnkv_counter_key.visits_all']:
        q4_stats.append(execute_query(q4_tmpl, date, table, 4))

CPU times: user 3.42 s, sys: 1.54 s, total: 4.97 s
Wall time: 4min 25s


In [62]:
show_stats(q4_stats)

median 1.06


table,counter_id,wo_counter_id,share
date,,,
2017-10-14,19.313653,18.146046,1.064345
2017-10-15,19.462037,15.300562,1.271982
2017-10-16,18.660111,18.120071,1.029803
2017-10-17,19.345077,18.681497,1.035521
2017-10-18,19.533524,18.658153,1.046916
2017-10-19,20.980774,18.897654,1.110232
2017-10-20,21.229891,18.650501,1.138301


In [63]:
df_to_wiki(show_stats(q4_stats).applymap(lambda x: round(x, 2)), True)

median 1.06
#|
 ||date | counter_id | wo_counter_id | share|| 
 ||2017-10-14 | 19.31 | 18.15 | 1.06|| 
 ||2017-10-15 | 19.46 | 15.3 | 1.27|| 
 ||2017-10-16 | 18.66 | 18.12 | 1.03|| 
 ||2017-10-17 | 19.35 | 18.68 | 1.04|| 
 ||2017-10-18 | 19.53 | 18.66 | 1.05|| 
 ||2017-10-19 | 20.98 | 18.9 | 1.11|| 
 ||2017-10-20 | 21.23 | 18.65 | 1.14|| 
 |#


## Query 5

In [24]:
q5_tmpl = '''

SELECT ((regionToCountry(region_id) AS country_id) = 0 ? region_id : country_id) AS country,
       ((arrayJoin(arrayFilter(r -> r IN (11193, 213, 11077, 10699, 11309, 10693, 225, 20525, 11119, 11443, 11111, 10897, 11266, 11117, 10950,
              11146, 10819, 10841, 120, 124, 125, 2, 10687, 11024, 11108, 11021, 11020, 11375, 54, 56, 11029, 10802, 66,
              983, 10233, 10231, 11131, 84, 29633, 29632, 29631, 29630, 29634, 11282, 20537, 20536, 20535, 20534, 20533,
              20532, 20531, 20530, 20539, 20538, 10705, 11084, 39, 38, 11004, 11353, 10904, 29629, 11457, 11450, 11095,
              20524, 10658, 20526, 20527, 20528, 20529, 21949, 10650, 11340, 65, 179, 10251, 10795, 11225, 172, 977,
              11156, 10853, 11153, 11015, 10995, 11010, 11158, 11012, 11013, 181, 187, 10857, 43, 10645, 11330, 168,
              20550, 20552, 10243, 11232, 11235, 96, 11069, 115, 10842, 117, 11148, 10926, 10712, 29413, 29412, 29411,
              29410, 29417, 29416, 29415, 29414, 10000, 157, 159, 10672, 20546, 20547, 20544, 20545, 20542, 20543,
              20540, 20541, 20548, 20549, 29404, 29406, 11079, 11403, 11176, 29403, 47, 11070, 29408, 29409, 11409, 1,
              10939, 10832, 10933, 144, 142, 143, 206, 209, 208, 10174, 10176, 149, 29407, 11318, 11316, 51, 11162,
              11398, 10772, 10776, 20551, 10946, 171, 11508, 11503, 11504, 103697, 11505, 11501, 167, 169, 207,  
              170), regionHierarchy(RegionID))) AS region_id) = country ? 
          -1 : region_id) AS region,
       arrayJoin([-1, platform_id]) AS platform,
       arrayJoin([-1, device_category_id, device_category_id IN (2, 3) ? 5 : 0]) AS device_category,
       arrayJoin([-1, browser_id IN (83, 72, 190) ? -7 : 0, browser_id IN (83, 72) ? -8 : 0, browser_id IN (83, 72, 2, 3) ? browser_id : 0]) AS browser,
       arrayJoin([-1, device_model_id, device_name_id]) AS device_model,
       arrayJoin(
           [
               search_engine_id,
               parent_search_engine_id,
               search_engine_id != 74 ? -1 : 0
           ]
       ) AS search_engine,
       -5 AS statistic_type,
       sum(searches) AS value
FROM (
SELECT RegionID,
       ((OSToRoot(OS) AS a) IN (107, 108, 103, 100) ? (a = 103 ? 106 : a) : 0) AS platform_id,
       (IsTablet ? 3 : (IsMobile ? 2 : (IsTV ? 4: 1))) AS device_category_id,
       (UserAgent IN (83, 72, 190, 2, 3) ? UserAgent : 0) AS browser_id,
       (MobilePhoneVendor in (7, 60, 63) ? MobilePhoneVendor : 0) AS device_model_id,
       arrayFirst((i, x) -> x,
                  [-13,
                   -14,
                   -15,
                   -16,
                   -17,
                   0
                  ],
                  [MobilePhoneModel = 'Galaxy J1',
                   MobilePhoneModel LIKE '%SM-J710F%',
                   MobilePhoneModel LIKE '%SM-G920F%',
                   MobilePhoneModel LIKE '%SM-G930F%',
                   MobilePhoneModel LIKE '%SM-J510FN%',
                   1
                  ]
                 ) as device_name_id,
       SearchEngineIDInternal,

       -- C 10 мая учитываем переходы с реферерами вида "android-app://com.google.android.googlequicksearchbox%" в трафикогенерацию Google.
       (SearchEngineIDInternal = 0 AND
        StartDate >= toDate('2016-05-11') AND
        Referer LIKE 'android-app://com.google.android.googlequicksearchbox%'
       ) AS condition_for_adding_visits_to_google,

       -- 11.05.2017 - 23.05.2017 учитываем переходы с реферерами вида "https://yandex.ru/search/news-cache/touch%" в трафикогенерацию Яндекс.Новостей.
       (StartDate BETWEEN '2017-05-11' AND '2017-05-23' AND
        Referer LIKE 'https://yandex.ru/search/news-cache/touch%'
       ) AS condition_for_adding_visits_to_news,

       -- До 30.11.2017 учитываем мобильные Яндекс.Картинки. METR-26988
       (StartDate <= '2017-11-30' AND
        domain(Referer) = 'yandex.ru' AND path(Referer) = '/clck/jsredir'
        AND IsMobile AND SearchEngineID = 13 
        AND ((extractURLParameter(Referer, 'from') AS RefererFrom) = 'yandex.ru;images%2Ftouch%2Fsearch;images;;'
          
        )
       ) AS condition_for_adding_yandex_images,

       transform(SearchEngineIDInternal, [2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 20, 40, 54, 68, 71, 72, 73, 85, 90, 96, 99, 101, 119, 121, 167, 179, 181, 185, 198, 211, 224, 225], [26, 27, 215, 35, 34, 100, 37, 36, 33, 26, 61, 122, 38, 27, 39, 35, 37, 84, 89, 95, 84, 100, 34, 33, 201, 36, 26, 184, 215, 122, 27, 26], 0) AS _parent_search_engine_id,
       (condition_for_adding_visits_to_google ? 27 : (
         condition_for_adding_visits_to_news or condition_for_adding_yandex_images ? 
          0 : _parent_search_engine_id)) AS parent_search_engine_id,
       (condition_for_adding_visits_to_google ? 224 : (
         condition_for_adding_visits_to_news ? 74 : (
          condition_for_adding_yandex_images ? 1 : SearchEngineIDInternal))) AS search_engine_id,

       -- Раскатка https на SERP'e
       ((protocol(StartURL) = 'http' AND parent_search_engine_id = 26 AND device_category_id = 2)
        ? arrayFirst((i, x) -> x,
                     [1.44,
                      3.79,
                      1.6,
                      1.355,
                      1.11,
                      0.62,
                      11.00,
                      2.00,
                      0.41,
                      0.45,
                      0.0
                     ],
                     [UserAgent = 72 AND (UserAgentMajor < 4
                                          OR (UserAgentMajor = 4
                                              AND toInt16OrZero(reinterpretAsString(UserAgentMinor)) < 4))
                                     AND StartDate >= toDate('2016-05-24'),
                      UserAgent = 37 AND UserAgentMajor <= 12 AND StartDate >= toDate('2016-05-24'),
                      UserAgent = 53 AND StartDate >= toDate('2016-12-14'),
                      UserAgent = 53 AND StartDate = toDate('2016-12-13'),
                      UserAgent = 53 AND StartDate >= toDate('2016-05-24'),
                      UserAgent = 44 AND StartDate >= toDate('2016-05-24'),
                      UserAgent = 1 AND StartDate >= toDate('2016-12-14'),
                      UserAgent = 1 AND StartDate = toDate('2016-12-13'),
                      UserAgent = 1 AND StartDate >= toDate('2016-05-24'),
                      UserAgent IN (6, 28, 100, 3, 0, 33, 57, 88, 15, 64, 5, 84, 23, 2, 4, 155, 142, 108, 63, 82, 86, 137, 79, 22, 89,
                     47, 110, 150, 30, 87, 90, 66, 61, 48, 18, 41, 85, 29, 91, 8, 81, 27, 17, 55, 113),
                      1
                     ]
                    )
        : 0.0) AS multiply_coefficient_http_serp,

       -- Сокрытие рефереров Google в Android Browser
       ((StartDate >= toDate('2016-06-23') AND parent_search_engine_id = 27 AND UserAgent = 72
         AND (UserAgentMajor < 4 OR (UserAgentMajor = 4 AND toInt16OrZero(reinterpretAsString(UserAgentMinor)) < 4)))
        ? arrayFirst((i, x) -> x,
                     [0.572,
                      0.867,
                      0.0
                     ],
                     [device_category_id = 2,
                      device_category_id = 3,
                      1
                     ]
                    )
        : 0.0) AS multiply_coefficient_android_browser_referal_hiding,

       -- Трафик из Поискового Приложения Google
       ((StartDate <= toDate('2016-05-10') AND parent_search_engine_id = 27 AND UserAgent in (83,72,190) AND platform_id == 108)
        ? (((IsTablet ? 0.17 : (IsMobile ? 0.26 : 0.0)) + transform(MobilePhoneVendor, [7, 63], [0.08, 0.04], -0.04) as coef) > 0
           ? coef : 0)
        : 0.0) AS multiply_coefficient_google_search_app,

       -- extdata1170 сокрытие рефереров Chrome Mobile
       ((parent_search_engine_id = 27 AND UserAgent = 83 AND platform_id == 108 and device_category_id == 2)
        ? arrayFirst((i, x) -> x,
                     [0.01,
                      0.02,
                      0.03,
                      0.04,
                      0.05,
                      0.06,
                      0.11,
                      0.14,
                      0.15,
                      0.135,
                      0.1127,
                      0.065,
                      0.0
                     ],
                     [toDate('2016-08-29') <= StartDate AND StartDate <= toDate('2016-09-04'),
                      toDate('2016-08-25') <= StartDate AND StartDate <= toDate('2016-08-28'),
                      toDate('2016-08-23') <= StartDate AND StartDate <= toDate('2016-08-24'),
                      toDate('2016-08-21') <= StartDate AND StartDate <= toDate('2016-08-22'),
                      toDate('2016-08-20') <= StartDate AND StartDate <= toDate('2016-08-20'),
                      toDate('2016-08-19') <= StartDate AND StartDate <= toDate('2016-08-19'),
                      toDate('2016-08-18') <= StartDate AND StartDate <= toDate('2016-08-18'),
                      toDate('2016-08-12') <= StartDate AND StartDate <= toDate('2016-08-17'),
                      toDate('2016-03-23') <= StartDate AND StartDate <= toDate('2016-08-11'),
                      toDate('2016-01-27') <= StartDate AND StartDate <= toDate('2016-03-22'),
                      toDate('2015-10-24') <= StartDate AND StartDate <= toDate('2016-01-26'),
                      toDate('2015-09-05') <= StartDate AND StartDate <= toDate('2015-10-23'),
                      1
                     ]
                    )
        : 0.0) as referal_hiding_raw,
       arrayFirst((i, x) -> x,
                  [referal_hiding_raw * 1.8,
                   referal_hiding_raw * 2.9,
                   referal_hiding_raw * 2.5,
                   referal_hiding_raw * 2.5,
                   referal_hiding_raw * 3.1,
                   referal_hiding_raw
                  ],
                  [device_name_id = -13,
                   device_name_id = -14,
                   device_name_id = -15,
                   device_name_id = -16,
                   device_name_id = -17,
                   1
                  ]
                 ) as multiply_coefficient_chrome_mobile_referal_hiding,

       -- RESEARCH-2286 сокрытие рефереров приложения Google в декабре 2016 - марте 2017
       ((parent_search_engine_id = 27 AND UserAgent = 83 AND platform_id == 108 and device_category_id == 2
         AND protocol(Referer) = 'https')
        ? (StartDate BETWEEN '2016-12-09' AND '2017-01-24' ? 0.112 : (
            StartDate BETWEEN '2017-01-25' AND '2017-03-03' ? 0.03 : 0
          )) * transform(regionToCountry(RegionID), [149, 187], [1.15, 0.8], 1)
        : 0) as multiply_coefficient_december16_chrome_mobile_referal_hiding,

       --- RESEARCH-2475 корректировка абсолютов Яндекса из за бустинга метричных сайтов
       ((StartDate >= toDate('2017-09-19') AND parent_search_engine_id=26)
        ? arrayFirst((i, x) -> x,
                     [0.9852,
                      0.9855,
                      0.9925,
                      0.9897,
                      0.9870,
                      0.9899,
                      0.9898,
                      0.9860,
                      0.9851,
                      0.9769,
                      0.9860,
                      0.9892,
                      1
                     ],
                     [IsMobile AND IsTablet AND regionToCountry(RegionID) = 225,
                      IsMobile AND IsTablet=0 AND regionToCountry(RegionID) = 225,
                      IsMobile = 0 AND regionToCountry(RegionID) = 225,
                      IsMobile = 0 AND regionToCountry(RegionID) = 187,
                      IsMobile = 0 AND regionToCountry(RegionID) = 159,
                      IsMobile = 0 AND regionToCountry(RegionID) = 149,
                      IsMobile AND IsTablet = 0 AND regionToCountry(RegionID) = 187,
                      IsMobile AND IsTablet = 0 AND regionToCountry(RegionID) = 159,
                      IsMobile AND IsTablet = 0 AND regionToCountry(RegionID) = 149,
                      IsMobile AND IsTablet AND regionToCountry(RegionID) = 187,
                      IsMobile AND IsTablet AND regionToCountry(RegionID) = 159,
                      IsMobile AND IsTablet AND regionToCountry(RegionID) = 149,
                      1
                     ]
                    )
        : 1) as multiply_coefficient_boost,

       --- RESEARCH-2471 корректировка из-за разблокировки счетчиков Метрики Оперой
       ((StartDate >= toDate('2016-05-16') AND StartDate<=toDate('2017-08-03') AND
         StartTime <= '2017-08-03 17:25:00' AND UserAgent = 2 AND device_category_id == 1)
        ? arrayFirst((i, x) -> x,
                     [(toDate('2017-04-08') - toDate('2016-05-15')) * 0.0006 + 1.012,
                      (toDate('2017-04-08') - toDate('2016-05-15')) * 0.000509 + 0.99949,
                      (toDate('2017-04-08') - toDate('2016-05-15')) * 0.0005 + 0.99,
                      (toDate('2017-04-08') - toDate('2016-05-15')) * 0.00052 + 1.1623
                     ],
                     [regionToCountry(RegionID) = 225,
                      regionToCountry(RegionID) = 159,
                      regionToCountry(RegionID) = 149,
                      regionToCountry(RegionID) = 187 AND StartDate >= toDate('2016-06-19')
                     ]
                    )
        : 1) as multiply_coefficient_opera,

       --- RESEARCH-2525 корректировка сокрытия рефереров Яндекса в десктопном Firefox
       ((StartDate BETWEEN '2017-09-27' AND '2017-10-12' AND StartTime <= '2017-10-12 13:05:00' AND
         parent_search_engine_id = 26 AND NOT IsMobile AND
         UserAgent = 3 AND UserAgentMajor = 56)
        ? transform(regionToCountry(RegionID),
                    [225,  149,  159,  983],
                    [0.36, 0.36, 0.36, 0.67],
                    0.36
                   )
        : 0.0) as multiply_coefficient_firefox,

       --- RESEARCH-2558 корректировка перераспределения http://google.ru/url рефереров гугла из десктоп в мобильное
       ((StartDate >= '2017-11-14' AND StartTime >= '2017-11-14 16:30:00'
         AND regionToCountry(RegionID) = 225 and parent_search_engine_id = 27
         AND domainWithoutWWW(Referer) IN ('google.com', 'google.ru') 
         AND protocol(Referer) = 'http' AND path(Referer) = '/url'
        )
        ? transform(UserAgent,
                    [6,     83,   70],
                    [1.673, 0.19, 0.343],
                    1.0
                   )
        : 1.0) as multiply_coefficient_google_http_url,

       ((multiply_coefficient_http_serp +
         multiply_coefficient_android_browser_referal_hiding +
         multiply_coefficient_google_search_app + 1.0) *
        (1.0 + multiply_coefficient_chrome_mobile_referal_hiding +
         multiply_coefficient_december16_chrome_mobile_referal_hiding) *
        (multiply_coefficient_boost) * multiply_coefficient_opera *
        (1 + multiply_coefficient_firefox) * 
        multiply_coefficient_google_http_url * sum(Sign)
       ) AS searches
FROM {table}
WHERE StartDate = toDate('{date}') AND IsYandex = 0
      AND (SearchEngineIDInternal != 0 OR condition_for_adding_visits_to_google)
      AND CounterID GLOBAL IN (SELECT * FROM remote('mtstat01-1', good_counters_daily.counters_20171208))
GROUP BY RegionID, platform_id, device_category_id, browser_id, device_model_id, device_name_id,
         SearchEngineIDInternal, condition_for_adding_visits_to_google, condition_for_adding_visits_to_news,
         condition_for_adding_yandex_images,
         multiply_coefficient_http_serp, multiply_coefficient_android_browser_referal_hiding, referal_hiding_raw,
         multiply_coefficient_google_search_app, multiply_coefficient_chrome_mobile_referal_hiding,
         multiply_coefficient_december16_chrome_mobile_referal_hiding,
         multiply_coefficient_boost, multiply_coefficient_opera, multiply_coefficient_firefox,
         multiply_coefficient_google_http_url
)
WHERE platform != 0 AND device_category != 0 AND browser != 0 AND device_model != 0 AND search_engine != 0
GROUP BY region_id, platform, device_category, browser, device_model, search_engine
HAVING value >= 200
'''

In [26]:
%%time
q5_stats = []

for date in pd.date_range('2017-10-08', '2017-10-14'):
    print date
    for table in ['visits_all', 'zgmnkv_counter_key.visits_all']:
        q5_stats.append(execute_query(q5_tmpl, date, table, 5))

2017-10-08 00:00:00
2017-10-09 00:00:00
2017-10-10 00:00:00
2017-10-11 00:00:00
2017-10-12 00:00:00
2017-10-13 00:00:00
2017-10-14 00:00:00
CPU times: user 1.48 s, sys: 508 ms, total: 1.98 s
Wall time: 13min 47s


In [28]:
show_stats(q5_stats)

median 1.49


table,counter_id,wo_counter_id,share
date,,,
2017-10-08,74.374660,47.836539,1.554767
2017-10-09,73.337774,49.256067,1.488908
2017-10-10,65.799995,46.988183,1.400352
2017-10-11,70.112726,46.938951,1.493700
2017-10-12,67.840016,51.410444,1.319577
2017-10-13,73.434098,47.590888,1.543029
2017-10-14,66.505303,45.835527,1.450955


In [65]:
df_to_wiki(show_stats(q5_stats).applymap(lambda x: round(x, 2)), True)

median 1.49
#|
 ||date | counter_id | wo_counter_id | share|| 
 ||2017-10-08 | 74.37 | 47.84 | 1.55|| 
 ||2017-10-09 | 73.34 | 49.26 | 1.49|| 
 ||2017-10-10 | 65.8 | 46.99 | 1.4|| 
 ||2017-10-11 | 70.11 | 46.94 | 1.49|| 
 ||2017-10-12 | 67.84 | 51.41 | 1.32|| 
 ||2017-10-13 | 73.43 | 47.59 | 1.54|| 
 ||2017-10-14 | 66.51 | 45.84 | 1.45|| 
 |#


## Query 6

In [31]:
q6_tmpl = '''
SELECT
    sum(visits) * 10 as visits_all,
    sumIf(visits,
          FirstDate <= '{date}' AND FirstDate >= toDate('{date}')-7
        ) * 10 as visits_7,
    sumIf(visits,
          FirstDate <= '{date}' AND FirstDate >= toDate('{date}')-30
        ) * 10 as visits_30,
    sumIf(visits,
          FirstDate <= '{date}' AND FirstDate >= '2000-01-01'
        ) * 10 as visits_known,
    device_model,
    -1 AS platform,
    device_category,
    search_engine,
    '{date}' as start_date
FROM
    (SELECT
        sum(Sign) as visits,
        UserID,
        arrayJoin([
            (MobilePhoneVendor IN (1, 7, 18, 83, 6, 12, 9, 17, 4, 16, 63, 229, 60, 0, 207, 400) ? MobilePhoneVendor : -6) AS device_vendor,
            arrayFirst((i, x) -> x,
                [-13,
                 -14,
                 -15,
                 -16,
                 -17,
                 0
                ],
                [MobilePhoneModel = 'Galaxy J1',
                 MobilePhoneModel LIKE '%SM-J710F%',
                 MobilePhoneModel LIKE '%SM-G920F%',
                 MobilePhoneModel LIKE '%SM-G930F%',
                 MobilePhoneModel LIKE '%SM-J510FN%',
                 1
                ]) as device_name_id
            ]) AS device_model,
        arrayJoin([-1, (IsTablet ? 3 : (IsMobile ? 2 : (IsTV ? 4: 1))) AS c,
            c IN (2, 3) ? 5 : 0]) AS device_category,
        arrayJoin(['_total_', transform(SearchEngineIDInternal, [2, 13, 181, 3, 68, 224],
            ['yandex', 'yandex', 'yandex', 'google', 'google', 'google'], 'other')])
                AS search_engine
    FROM {table}
    SAMPLE 1 / 10
    WHERE StartDate == toDate('{date}') AND regionToCountry(RegionID) = 225
        AND IsYandex = 0 AND IsMobile = 1 AND SearchEngineIDInternal NOT IN (0, 74)
        AND CounterID GLOBAL IN (SELECT * FROM remote('mtstat01-1', good_counters_daily.counters_20171207))
        AND device_category IN (2, 3, 5)
        AND search_engine IN ('_total_', 'yandex', 'google')
    GROUP BY
        device_category,
        device_model,
        search_engine,
        UserID)
ANY LEFT JOIN
    (SELECT
        UserID,
        FirstDate
    FROM remote('mtstat01-1', statbox.uid_day_crypta)
    SAMPLE 1 / 10)
USING UserID
GROUP BY
    device_category,
    device_model,
    search_engine,
    platform,
    start_date
HAVING visits_all >= 200
'''

In [37]:
%%time
q6_stats = []

for date in pd.date_range('2017-10-08', '2017-10-14'):
    print date
    for table in ['visits_all', 'zgmnkv_counter_key.visits_all']:
        q6_stats.append(execute_query(q6_tmpl, date, table, 6))

2017-10-08 00:00:00
2017-10-09 00:00:00
2017-10-10 00:00:00
2017-10-11 00:00:00
2017-10-12 00:00:00
2017-10-13 00:00:00
2017-10-14 00:00:00
CPU times: user 72 ms, sys: 24 ms, total: 96 ms
Wall time: 12min 15s


In [38]:
show_stats(q6_stats)

median 0.82


table,counter_id,wo_counter_id,share
date,,,
2017-10-08,49.009986,59.985433,0.817031
2017-10-09,50.928937,61.959855,0.821967
2017-10-10,46.779662,54.914156,0.851869
2017-10-11,45.461448,57.260920,0.793935
2017-10-12,47.228474,57.647212,0.819267
2017-10-13,45.783476,54.400626,0.841598
2017-10-14,48.018356,56.321558,0.852575


In [66]:
df_to_wiki(show_stats(q6_stats).applymap(lambda x: round(x, 2)), True)

median 0.82
#|
 ||date | counter_id | wo_counter_id | share|| 
 ||2017-10-08 | 49.01 | 59.99 | 0.82|| 
 ||2017-10-09 | 50.93 | 61.96 | 0.82|| 
 ||2017-10-10 | 46.78 | 54.91 | 0.85|| 
 ||2017-10-11 | 45.46 | 57.26 | 0.79|| 
 ||2017-10-12 | 47.23 | 57.65 | 0.82|| 
 ||2017-10-13 | 45.78 | 54.4 | 0.84|| 
 ||2017-10-14 | 48.02 | 56.32 | 0.85|| 
 |#
